# Desafio – Parte 2 (DES_2)

A princípio apliquei técnicas que usavam somente **a topologia da rede** para determinar as ligações dos nós a serem avaliados, mas com essas técnicas não obtive uma boa acurácia.
Alguns exemplos que testei:
- Node2Vec
- Common Neighbors
- Jaccard Coefficient
- Preferential Attachment
- Random Walks

Consegui resultados relativamente bons com alguns desses algoritmos, mas como queria usar os atributos dos nós e das arestas implementei uma solução usando Redes Neurais. Comecei testando somente as features de distância geográfica, o que ficou bem próximo de alguns dos algoritmos que citei anteriormente.
Ao longo do processo, fui melhorando o treinamento com mais features e, no fim, utilizei todos os atributos dos nós e o da aresta sobre as features:

- Distance: foi calculada a partir dos atributos latitude e longitude, a feature é o valor em quilômetros da distância dois nós;
- Category Intersection: a intersecção entre as categorias dos nós;
- Category Union: a união entre as categorias dos nós;
- Stars: não foi feita nenhuma tratativa sobre esse dado em relação ao que existe nos próprios nós, somente os valores de cada nó foram adicionados no array;
- Review Count: não foi feita nenhuma tratativa sobre esse dado em relação ao que existe nos próprios, somente os valores de cada nó foram adicionados no array;
- Edge Weight: um valor 0 ou 1 indicando se peso da aresta é “relevante” ou não. Para esse caso foi feita uma validação para considerar que a aresta é relevante apenas se ela tiver um peso maior que 3, pois a grande maioria da rede é menor que isso, o que causou problemas quando treinei o modelo.

No fim, notei que nem todas essas features causaram um grande efeito no treinamento do modelo. As que mais se destacaram foram as de categoria: Intersection e Union, stars e review count.
Já a distância e o peso da aresta não mudaram muito a acurácia, mas ajudaram a chegar na melhor previsão que tive.
Quando o modelo foi treinado usando somente as ligações existentes, a acurácia não teve muito sucesso, mas, ao adicionar features entre nós que não se ligam, consegui treinar o modelo da maneira desejada. Ao definir a classe do dataset, o len ficou a quantidade de arestas * 2 justamente para metade do treinamento ter dados de nós que não se conectam.
```python
    def __len__(self):
        return len(self.edges) * 2

    def __getitem__(self, idx):
        if idx < len(self.edges):
            node1, node2 = self.edges[idx]
            label = int(self.graph.has_edge(node1, node2))
        else:
            node1, node2 = self.random_node, self.random_node
            while self.graph.has_edge(node1, node2):
                node1, node2 = self.random_node, self.random_node
            label = 0

        features = extract_features(node1=node1, node2=node2, label=label)
        return features, label
```
Em relação ao treinamento do modelo, usei os seguintes hiperparâmetros:
- learning_rate = 0.001
- num_epochs = 1000
- criterion = nn.BCELoss()
- optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

Esses valores foram testados e alterados por várias iterações de acurácia e foi com eles que obtive um melhor resultado. Alguns exemplos de outros parâmetros que usei:

- **criterion**:
    - MSELoss
    - CTCLoss
    - CrossEntropyLoss
- **optimizer**:
    - Adam
    - LBFGS
    - SGD
- **learning_rate**:
    - 0,1
    - 0,01
    - 0,0001
- **num_epochs**:
    - 10
    - 100
    - 10000
    - 100000

Critetion, optimizer e learning_rate foram os que mais mudaram a acurácia dos resultados. Já as epochs pouco influenciaram, mesmo em baixos e altos valores (10~100000).

A acurácia ficou em torno de <code>0.80~0.82</code> e os dois melhores resultados que obtive no Kaggle foram: <code>0.82666 e 0.82333</code>. Os CSVs com os dois melhores resultados do Kaggle estão no diretório <code>kaggle_results</code>:
```
desafio-parte-2
└── kaggle_results
    ├── resultados__0.82333.csv
    └── resultados__0.82666.csv
```
Por fim, a solução utilizou redes neurais e corresponde ao código que envio nesse desafio.

Para instalar as dependências localmente e rodar o notebook utilize o poetry e python 3.11.
```bash
pip install -U pip
pip install poetry
poetry install
```

Isso vai criar uma virtualenv para que seja usado no kernel do Jupyter.